In [1]:
import pandas as pd
import re
from pymystem3 import Mystem
import gensim as gs
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, GRU,SimpleRNN

In [27]:
pos = pd.read_csv("positive.csv", header=None, error_bad_lines=False, sep=';', names = [str(x) for x in (list(range(3)) + ['Text'] + list(range(4,12)))])
pos["emotion"] = 1
pos.drop([str(x) for x in (list(range(3)) + list(range(4,12)))], inplace=True, axis=1)
pos.head()

,Text,emotion
0,"@first_timee хоть я и школота, но поверь, у на...",1
1,"Да, все-таки он немного похож на него. Но мой ...",1
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1


In [29]:
neg = pd.read_csv("negative.csv", header=None, error_bad_lines=False, sep=';', names = [str(x) for x in (list(range(3)) + ['Text'] + list(range(4,12)))])
neg["emotion"] = 0
neg.drop([str(x) for x in (list(range(3)) + list(range(4,12)))], inplace=True, axis=1)
neg.head()

,Text,emotion
0,на работе был полный пиддес :| и так каждое за...,0
1,"Коллеги сидят рубятся в Urban terror, а я из-з...",0
2,@elina_4post как говорят обещаного три года жд...,0
3,"Желаю хорошего полёта и удачной посадки,я буду...",0
4,"Обновил за каким-то лешим surf, теперь не рабо...",0


In [ ]:
df = pd.concat([neg.iloc[:2000], pos.iloc[:2000]], ignore_index=True)

In [37]:
reg = re.compile('[^а-яА-ЯёЁ ]')
df["cleaned"] = df["Text"].apply(lambda x:re.sub(r'\s+', ' ', (reg.sub(' ', x))).strip())

In [94]:
def sentenceInLemmas(sentence, m=Mystem()):
    processed = m.analyze(sentence)
    tagged_sentences = []
    tagged = []
    for w in processed:        
        if w["text"].strip() == "+":
            tagged_sentences.append(" ".join(tagged))
            tagged = []
            continue
        try:
            lemma = w["analysis"][0]["lex"].lower().strip()
            pos = w["analysis"][0]["gr"].split(',')[0]
            pos = pos.split('=')[0].strip()
            tagged.append(lemma+'_'+uTag(pos))
        except:
            continue
    tagged_sentences.append(" ".join(tagged))
    return tagged_sentences


def uTag(tag):
    return {
        'A':       'ADJ',                                                                                                                                                                                                                                                                  
        'ADV':     'ADV',                                                                                                                                                                                                                                                                  
        'ADVPRO':  'ADV',                                                                                                                                                                                                                                                                  
        'ANUM':    'ADJ',                                                                                                                                                                                                                                                                    
        'APRO':    'DET',                                                                                                                                                                                                                                                                    
        'COM':     'ADJ',                                                                                                                                                                                                                                                                  
        'CONJ':    'SCONJ',                                                                                                                                                                                                                                                                 
        'INTJ':    'INTJ',                                                                                                                                                                                                                                                                 
        'NONLEX':  'X',                                                                                                                                                                                                                                                                      
        'NUM':     'NUM',                                                                                                                                                                                                                                                                   
        'PART':    'PART',                                                                                                                                                                                                                                                                   
        'PR':      'ADP',                                                                                                                                                                                                                                                                    
        'S':       'NOUN',                                                                                                                                                                                                                                                                   
        'SPRO':    'PRON',                                                                                                                                                                                                                                                                  
        'UNKN':    'X',                                                                                                                                                                                                                                                                      
        'V':       'VERB'
    }[tag]

In [98]:
text = ' + '.join(df["cleaned"].values)
df["lemmed"] = sentenceInLemmas(text)

In [102]:
df.head()

,Text,emotion,cleaned,lemmed
0,на работе был полный пиддес :| и так каждое за...,0,на работе был полный пиддес и так каждое закры...,на_ADP работа_NOUN быть_VERB полный_ADJ пиддес...
1,"Коллеги сидят рубятся в Urban terror, а я из-з...",0,Коллеги сидят рубятся в а я из за долбанной ви...,коллега_NOUN сидеть_VERB рубиться_VERB в_ADP а...
2,@elina_4post как говорят обещаного три года жд...,0,как говорят обещаного три года ждут,как_SCONJ говорить_VERB обещаной_ADJ три_NUM г...
3,"Желаю хорошего полёта и удачной посадки,я буду...",0,Желаю хорошего полёта и удачной посадки я буду...,желать_VERB хороший_ADJ полет_NOUN и_SCONJ уда...
4,"Обновил за каким-то лешим surf, теперь не рабо...",0,Обновил за каким то лешим теперь не работает п...,обновлять_VERB за_ADP какой_DET то_SCONJ леший...


In [ ]:
df.to_csv("prepared.csv", sep=";", header=None)

In [109]:
def to_matrix(frame, vocabsize, vocab):
    matrixes = []
    tone = []
    for i in tqdm_notebook(range(frame.values.shape[0])):
        try:
            words = frame.iloc[i]['lemmed'].split(' ')
            if len(words) == 0:
                break
            matr = np.zeros(shape=(1, vocabsize))
            for w in words:
                try:
                    matr = np.vstack((matr, vocab[w]))
                except KeyError:
                    pass
            if matr.shape[0] > 2:
                matrixes.append(matr[1:matr.shape[0]][:])
                tone.append(frame.iloc[i]['emotion'])
        except AttributeError:
            print(frame.iloc[i]['lemmed'])

    return matrixes, tone

In [121]:
vocab = gs.models.KeyedVectors.load_word2vec_format('web.bin', binary=True)
vocabsize = 300
timesteps = 32
batch_size = 64
n_epochs = 200
n_parts = 5

In [112]:
matrices, tones = to_matrix(df, vocabsize, vocab)

In [123]:
for i in tqdm_notebook(range(len(matrices))):
    matrices[i] = np.vstack(
        (np.random.normal(scale=0.005, size=(timesteps - matrices[i].shape[0], vocabsize)),matrices[i]))
matrices = np.array(matrices)

In [124]:
X_train, X_test, y_train, y_test = train_test_split(matrices, tones, test_size=0.2, random_state=42,stratify=tones)

In [126]:
y_train = np.array(y_train, ndmin=2).T
y_test = np.array(y_test, ndmin=2).T

In [130]:
def simple_rnn(timesteps, vocabsize):
    model = Sequential()
    model.add(SimpleRNN(1, return_sequences=False, input_shape=(timesteps, vocabsize), dropout=0.05, activation="sigmoid"))
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [131]:
def gru(timesteps, vocabsize):
    model = Sequential()
    model.add(GRU(1, return_sequences=False, input_shape=(timesteps, vocabsize), dropout=0.05, activation="sigmoid"))
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [132]:
def lstm(timesteps, vocabsize):
    model = Sequential()
    model.add(LSTM(1, return_sequences=False, input_shape=(timesteps, vocabsize), dropout=0.05, activation="sigmoid"))
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    return model

In [139]:
model1 = simple_rnn(timesteps, vocabsize)

In [140]:
%%time
model1.fit(X_train, y_train, batch_size=batch_size, epochs=50, verbose=2,validation_data=(X_test,y_test))
#Epoch 50/50
# - 1s - loss: 0.6349 - acc: 0.6432 - val_loss: 0.6548 - val_acc: 0.6073
#Wall time: 1min 7s

Train on 3055 samples, validate on 764 samples
Epoch 1/50
 - 2s - loss: 0.7248 - acc: 0.4985 - val_loss: 0.7099 - val_acc: 0.4987
Epoch 2/50
 - 1s - loss: 0.7013 - acc: 0.5087 - val_loss: 0.6949 - val_acc: 0.5380
Epoch 3/50
 - 1s - loss: 0.6905 - acc: 0.5342 - val_loss: 0.6881 - val_acc: 0.5524
Epoch 4/50
 - 1s - loss: 0.6843 - acc: 0.5565 - val_loss: 0.6845 - val_acc: 0.5733
Epoch 5/50
 - 1s - loss: 0.6807 - acc: 0.5728 - val_loss: 0.6818 - val_acc: 0.5851
Epoch 6/50
 - 1s - loss: 0.6782 - acc: 0.5764 - val_loss: 0.6796 - val_acc: 0.5864
Epoch 7/50
 - 1s - loss: 0.6750 - acc: 0.5869 - val_loss: 0.6777 - val_acc: 0.5851
Epoch 8/50
 - 1s - loss: 0.6726 - acc: 0.5899 - val_loss: 0.6761 - val_acc: 0.5812
Epoch 9/50
 - 2s - loss: 0.6709 - acc: 0.5938 - val_loss: 0.6746 - val_acc: 0.5785
Epoch 10/50
 - 1s - loss: 0.6687 - acc: 0.5964 - val_loss: 0.6732 - val_acc: 0.5851
Epoch 11/50
 - 1s - loss: 0.6669 - acc: 0.6010 - val_loss: 0.6721 - val_acc: 0.5838
Epoch 12/50
 - 1s - loss: 0.6659 - acc

In [141]:
model2 = gru(timesteps, vocabsize)

In [142]:
%%time
model2.fit(X_train, y_train, batch_size=batch_size, epochs=50, verbose=2,validation_data=(X_test,y_test))
#Epoch 50/50
# - 2s - loss: 0.6202 - acc: 0.6625 - val_loss: 0.6362 - val_acc: 0.6427
#Wall time: 1min 26s

Train on 3055 samples, validate on 764 samples
Epoch 1/50
 - 2s - loss: 0.6902 - acc: 0.5444 - val_loss: 0.6864 - val_acc: 0.6047
Epoch 2/50
 - 2s - loss: 0.6863 - acc: 0.5804 - val_loss: 0.6835 - val_acc: 0.6021
Epoch 3/50
 - 2s - loss: 0.6829 - acc: 0.5863 - val_loss: 0.6807 - val_acc: 0.5969
Epoch 4/50
 - 2s - loss: 0.6797 - acc: 0.5957 - val_loss: 0.6783 - val_acc: 0.5995
Epoch 5/50
 - 2s - loss: 0.6774 - acc: 0.6007 - val_loss: 0.6760 - val_acc: 0.6034
Epoch 6/50
 - 2s - loss: 0.6746 - acc: 0.5967 - val_loss: 0.6738 - val_acc: 0.6073
Epoch 7/50
 - 2s - loss: 0.6716 - acc: 0.6108 - val_loss: 0.6718 - val_acc: 0.6060
Epoch 8/50
 - 2s - loss: 0.6693 - acc: 0.6085 - val_loss: 0.6700 - val_acc: 0.6073
Epoch 9/50
 - 2s - loss: 0.6667 - acc: 0.6137 - val_loss: 0.6682 - val_acc: 0.6073
Epoch 10/50
 - 2s - loss: 0.6646 - acc: 0.6193 - val_loss: 0.6667 - val_acc: 0.6099
Epoch 11/50
 - 2s - loss: 0.6633 - acc: 0.6134 - val_loss: 0.6652 - val_acc: 0.6099
Epoch 12/50
 - 2s - loss: 0.6611 - acc

In [133]:
model3 = lstm(timesteps, vocabsize)

In [137]:
%%time
model3.fit(X_train, y_train, batch_size=batch_size, epochs=50, verbose=2,validation_data=(X_test,y_test))
#Epoch 50/50
# - 2s - loss: 0.5809 - acc: 0.7011 - val_loss: 0.6324 - val_acc: 0.6466
#Wall time: 1min 27s#

Train on 3055 samples, validate on 764 samples
Epoch 1/50
 - 2s - loss: 0.6066 - acc: 0.6727 - val_loss: 0.6365 - val_acc: 0.6492
Epoch 2/50
 - 2s - loss: 0.6060 - acc: 0.6740 - val_loss: 0.6363 - val_acc: 0.6479
Epoch 3/50
 - 2s - loss: 0.6038 - acc: 0.6812 - val_loss: 0.6360 - val_acc: 0.6492
Epoch 4/50
 - 2s - loss: 0.6038 - acc: 0.6753 - val_loss: 0.6360 - val_acc: 0.6492
Epoch 5/50
 - 2s - loss: 0.6032 - acc: 0.6769 - val_loss: 0.6358 - val_acc: 0.6492
Epoch 6/50
 - 2s - loss: 0.6028 - acc: 0.6756 - val_loss: 0.6355 - val_acc: 0.6505
Epoch 7/50
 - 2s - loss: 0.6008 - acc: 0.6795 - val_loss: 0.6354 - val_acc: 0.6518
Epoch 8/50
 - 2s - loss: 0.6009 - acc: 0.6750 - val_loss: 0.6356 - val_acc: 0.6466
Epoch 9/50
 - 2s - loss: 0.5997 - acc: 0.6812 - val_loss: 0.6358 - val_acc: 0.6479
Epoch 10/50
 - 2s - loss: 0.6005 - acc: 0.6815 - val_loss: 0.6351 - val_acc: 0.6531
Epoch 11/50
 - 2s - loss: 0.5995 - acc: 0.6809 - val_loss: 0.6355 - val_acc: 0.6479
Epoch 12/50
 - 2s - loss: 0.5990 - acc